In [1]:
import numpy as np
import time as tm
from sklearn.metrics import mean_absolute_error as mae

In [2]:
X_trn = np.loadtxt( "train_challenges.dat" )
y_trn = np.loadtxt( "train_responses.dat" )
X_tst = np.loadtxt( "dummy/dummy_test_challenges.dat" )
y_tst = np.loadtxt( "dummy/dummy_test_responses.dat" )
w_gold = np.loadtxt( "dummy/dummy_w.dat" )

In [3]:
# Hard Thresholding operation
# Given a vector and a sparsity level, sparsify the vector
# by setting small magnitude coordinates to 0
def HT( v, k ):
    t = np.zeros_like( v )
    if k < 1:
        return t
    else:
        ind = np.argsort( abs( v ) )[ -k: ]
        t[ ind ] = v[ ind ]
        return t

In [4]:

def my_fit( X_trn, y_trn ):

    w = np . zeros_like ( X_trn [0])
    v = np . ones_like ( X_trn [0])
    t =0.3 # step length
    lamda =0.01 # regularization constant
    for j in range (100) :
        for i in range (1600) :
            sgt = lamda * v +(1/1600 * X_trn [ i ]) * ( np . dot ( X_trn [ i ] , w . T ) - y_trn [ i ] )
            w = w - t * sgt
        w = HT (w , 512) # our model sparsity is 512
        t = t /1.02
    return w


In [5]:
def supp_int( x, y, k ):
    x_t = HT( x, k )
    y_t = HT( y, k )
    supp_x = np.where( x_t != 0, 1, 0 )
    supp_y = np.where( y_t != 0, 1, 0 )
    return np.dot( supp_x, supp_y ) / k

In [6]:
n_trials = 5

t_train = 0
model_err = 0
mae_err = 0
supp_rec = 0

S = 512

In [7]:
for t in range( n_trials ):
  tic = tm.perf_counter()
  w_hat = HT( my_fit( X_trn, y_trn ), S )
  toc = tm.perf_counter()
  t_train += toc - tic
  model_err += np.linalg.norm( w_hat - w_gold )
  mae_err += mae( y_tst, X_tst @ w_hat )
  supp_rec += supp_int( w_hat, w_gold, S )

In [8]:
t_train /= n_trials
model_err /= n_trials
mae_err /= n_trials
supp_rec /= n_trials

print( t_train, model_err, mae_err, supp_rec )

2.7610829200013542 226.2420673922405 43.82718352924236 0.2578125
